### 소방청 - 경기도 소방기관 추출

In [2]:
import time 
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.2967198&myY=127.0078127&searchKeyword='

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
driver.get(url)

In [5]:
# 검색 입력상자 찾아서 경기도 입력하기
driver.find_element(By.ID, 'searchKeyword').send_keys('경기도')

In [6]:
# 검색 버튼 찾아서 클릭하기
driver.find_element(By.ID, 'fsSearchBtn').click()

In [7]:
#페이지 이동해서 HTML 코드를 BeautifulSoup으로 parsing
soup =BeautifulSoup(driver.page_source,'html.parser')
lis = soup.select('.stations-list > li')
len(lis) # 10건

10

In [8]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
name

'군포소방서-송정-119 안전센터'

In [9]:
addr = li.select_one('address').get_text().strip()
addr

'경기도 군포시 산본동1156'

In [10]:
tel = li.select_one('.tel').get_text().strip()
tel

'031-479-8503'

In [11]:
lines = []
for li in lis:
    name = li.select_one('.title').get_text().strip()
    addr = li.select_one('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append({'관서명':name, '주소':addr, '전화번호':tel})
df = pd.DataFrame(lines)
df

,관서명,주소,전화번호
0,군포소방서-송정-119 안전센터,경기도 군포시 산본동1156,031-479-8503
1,부천소방서-원종-119 안전센터,경기도 부천시 원종로79번길 41 (원종동),032-650-4554
2,부천소방서-괴안-119 안전센터,경기도 부천시 부광로8번길 49 (괴안동),032-650-4544
3,부천소방서-중동-119 안전센터,경기도 부천시 부일로 346 (중동),032-650-4534
4,부천소방서-내동-119 안전센터,경기도 부천시 신흥로 364 (내동),032-650-4524
5,부천소방서-중앙-119 안전센터,경기도 부천시 신흥로 115 (중동),032-650-4504
6,분당소방서-구미-119 안전센터,경기도 성남시 분당구 돌마로 138 (구미동),031-8018-3551
7,분당소방서-야탑-119 안전센터,경기도 성남시 분당구 양현로 485 (야탑4동),031-8018-3531
8,분당소방서-수내-119 안전센터,경기도 성남시 분당구 백현로 241 (수내동),031-8018-3521
9,분당소방서-판교-119 안전센터,경기도 성남시 분당구 대왕판교로644번길 62 (삼평동),031-8018-3511


- 두 번째 페이지로 이동

In [12]:
# copy xpath
# //*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

- next(>) page로 이동

In [14]:
driver.find_element(By.CLASS_NAME, 'next_page').click()

- 모든 페이지에 대해서 적용(단, 이동은 selenium으로)

In [15]:
# 첫번째 페이지
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('경기도')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [16]:
# 검색결과 건수 확인
count = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/div/p/strong[2]').text
count = int(count[:-1])

In [17]:
import math
total_page = int(math.ceil(count / 10))
total_page


10

In [18]:
lines = []
for i in range(total_page):
    if i % 2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)
    if i > 0 and i % 2 == 0:
        driver.find_element(By.CLASS_NAME, 'next_page').click()
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.select_one('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append({'관서명':name, '주소':addr, '전화번호':tel})

In [19]:
driver.close()

In [20]:
df = pd.DataFrame(lines)
df.tail()

,관서명,주소,전화번호
95,의정부소방서-금오-119 안전센터,경기도 의정부시 호국로 1434-14,031-849-7510
96,의정부소방서-중앙-119 안전센터,경기도 의정부시 의정로 48,031-849-7500
97,일산소방서-대화-119 안전센터,경기도 고양시 일산서구 대화동 2321-2,031-930-0572
98,일산소방서-고봉-119 안전센터,경기도 고양시 일산동구 성현로 429,031-930-0551
99,일산소방서-중산-119 안전센터,경기도 고양시 일산동구 중산로 218,031-930-0541


In [21]:
df.head()

,관서명,주소,전화번호
0,군포소방서-송정-119 안전센터,경기도 군포시 산본동1156,031-479-8503
1,부천소방서-원종-119 안전센터,경기도 부천시 원종로79번길 41 (원종동),032-650-4554
2,부천소방서-괴안-119 안전센터,경기도 부천시 부광로8번길 49 (괴안동),032-650-4544
3,부천소방서-중동-119 안전센터,경기도 부천시 부일로 346 (중동),032-650-4534
4,부천소방서-내동-119 안전센터,경기도 부천시 신흥로 364 (내동),032-650-4524
